In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

In [2]:
# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 数据预处理和增强
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# 加载数据
data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [5]:
# 定义模型
model = models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

C:\Users\吳欣紘\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\吳欣紘\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\吳欣紘/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100.0%


In [6]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
# 训练模型
num_epochs = 25

def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 每个epoch有两个阶段：训练阶段和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 设置模型为训练模式
            else:
                model.eval()   # 设置模型为评估模式

            running_loss = 0.0
            running_corrects = 0

            # 遍历数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 前向传播
                # 只在训练阶段计算梯度
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 反向传播和优化，只在训练阶段进行
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    return model

model = train_model(model, criterion, optimizer, num_epochs)


# 在测试集上评估模型
model.eval()
running_corrects = 0
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / dataset_sizes['test']
print(f'Test Acc: {test_acc:.4f}')

Epoch 0/24
----------
train Loss: 0.5353 Acc: 0.7992
val Loss: 0.4945 Acc: 0.8100

Epoch 1/24
----------
train Loss: 0.3087 Acc: 0.8783
val Loss: 0.4524 Acc: 0.8100

Epoch 2/24
----------
train Loss: 0.2734 Acc: 0.9029
val Loss: 0.4472 Acc: 0.8267

Epoch 3/24
----------
train Loss: 0.2308 Acc: 0.9167
val Loss: 0.2472 Acc: 0.9000

Epoch 4/24
----------
train Loss: 0.2177 Acc: 0.9183
val Loss: 0.2832 Acc: 0.8933

Epoch 5/24
----------
train Loss: 0.2167 Acc: 0.9229
val Loss: 0.2983 Acc: 0.8833

Epoch 6/24
----------
train Loss: 0.2033 Acc: 0.9263
val Loss: 0.1815 Acc: 0.9300

Epoch 7/24
----------
train Loss: 0.1827 Acc: 0.9325
val Loss: 0.2165 Acc: 0.9300

Epoch 8/24
----------
train Loss: 0.1509 Acc: 0.9421
val Loss: 0.2509 Acc: 0.9267

Epoch 9/24
----------
train Loss: 0.1613 Acc: 0.9379
val Loss: 0.2262 Acc: 0.9300

Epoch 10/24
----------
train Loss: 0.1299 Acc: 0.9483
val Loss: 0.3058 Acc: 0.8900

Epoch 11/24
----------
train Loss: 0.1503 Acc: 0.9392
val Loss: 0.1624 Acc: 0.9367

Ep